Author: Kyle Herbruger
Date: 10/17/2023

Program loads in two .csv files, with the xy information from the OpenCV hand tracker. Then plots them, and converts to a gif.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

In [213]:
class handPlotter():
    def __init__(self, dataX, dataY, dataZ, nameVar = ''):
        self.dataX = dataX
        self.dataY = dataY
        self.dataZ = dataZ
        self.nameVar = nameVar
        
        leng = len(dataX)
        #dataY = dataY - dataY.max()
        dataZ = dataZ * -1
        
#         clr = dataZ
#         clr = clr/dataZ.max()
#         for ia in clr:
#             ia = [ia, ia, ia]
        
        badFrameCount = 0.0
        zMax = max(np.max(dataZ), abs(np.min(dataZ)))
        frames = []
        # Generating plots
        for ia in range(leng):
            # Only save frame if there is data
            # Make plot with one frame
            clrZ = np.array(dataZ[ia])/zMax
            plt.plot(dataX[ia,0:5],dataY[ia,0:5], 'ro-')
            plt.plot(dataX[ia,5:9],dataY[ia,5:9], 'ro-')
            plt.plot(dataX[ia,9:13],dataY[ia,9:13], 'ro-')
            plt.plot(dataX[ia,13:17],dataY[ia,13:17], 'ro-')
            plt.plot(dataX[ia,17:21],dataY[ia,17:21], 'ro-')
            plt.plot([dataX[ia,0],dataX[ia,5],dataX[ia,9],dataX[ia,13],dataX[ia,17],dataX[ia,0]],
                    [dataY[ia,0],dataY[ia,5],dataY[ia,9],dataY[ia,13],dataY[ia,17],dataY[ia,0]], 'ro-')
            plt.scatter(dataX[ia,0:21],dataY[ia,0:21], s=clrZ[0:21]*30+25, c=clrZ[0:21],cmap='Oranges')
            plt.plot(dataX[ia,21:26],dataY[ia,21:26], 'bo-')
            plt.plot(dataX[ia,26:30],dataY[ia,26:30], 'bo-')
            plt.plot(dataX[ia,30:34],dataY[ia,30:34], 'bo-')
            plt.plot(dataX[ia,34:38],dataY[ia,34:38], 'bo-')
            plt.plot(dataX[ia,38:42],dataY[ia,38:42], 'bo-')
            plt.plot([dataX[ia,21],dataX[ia,26],dataX[ia,30],dataX[ia,34],dataX[ia,38],dataX[ia,21]],
                    [dataY[ia,21],dataY[ia,26],dataY[ia,30],dataY[ia,34],dataY[ia,38],dataY[ia,21]], 'bo-')
            plt.scatter(dataX[ia,22:42],dataY[ia,22:42], s=clrZ[22:42]*30+25, c=clrZ[22:42],cmap='Purples')
            # Invert Y-Axis. Idk why it's originally saved upside down :/
            plt.ylim(dataY[ia].max() + 100, dataY[ia].min() - 100)
            plt.xlim(0, dataX[ia].max() + 100)
            plt.xticks([])
            plt.yticks([])
            plt.savefig(f'./LR_CSV/img/img_{nameVar}{ia}.png',transparent=False,facecolor='white')
            plt.close()
            imageb = imageio.v2.imread(f'./LR_CSV/img/img_{nameVar}{ia}.png')
            frames.append(imageb)
            
            if (ia % 50) == 0:
                print('Processing...', (ia/leng)*100, '%')

        print('Schemeing...')
        imageio.mimsave(f'./LR_CSV/giffy2_{nameVar}.gif',
                       frames,
                       fps=20)
        print('Scheme complete >:)')
        print('Bad Frame Ratio: ', (badFrameCount / leng)*100)

In [214]:
def main():
    fileLoc = './LR_CSV/sng'
    booster = np.ones(21)
    booster[4] = 4
    booster[8] = 4
    booster[12] = 4
    booster[16] = 4
    booster[20] = 4
    dataX1 = np.loadtxt(f'{fileLoc}RightX.csv', delimiter=",", dtype=int)
    dataY1 = np.loadtxt(f'{fileLoc}RightY.csv', delimiter=",", dtype=int)
    dataZ1 = np.loadtxt(f'{fileLoc}RightZ.csv', delimiter=",", dtype=int)
    for ia in range(len(dataZ1)):
        dataZ1[ia] = dataZ1[ia] * booster
#     handPlotter(dataX, dataY, dataZ, 'R_a')

    dataX = np.ones((len(dataX1), 42))
    dataY = np.ones((len(dataX1), 42))
    dataZ = np.ones((len(dataX1), 42))
    
    dataX2 = ((np.loadtxt(f'{fileLoc}LeftX.csv', delimiter=",", dtype=int)))
    dataY2 = ((np.loadtxt(f'{fileLoc}LeftY.csv', delimiter=",", dtype=int)))
    dataZ2 = ((np.loadtxt(f'{fileLoc}LeftZ.csv', delimiter=",", dtype=int)))
    for ia in range(len(dataZ1)):
        dataZ2[ia] = dataZ2[ia] * booster
    
    for ia in range(len(dataX)):
        dataX[ia] = np.concatenate((dataX2[ia],dataX1[ia]))
        dataY[ia] = np.concatenate((dataY2[ia],dataY1[ia]))
        dataZ[ia] = np.concatenate((dataZ2[ia],dataZ1[ia]))
    handPlotter(dataX, dataY, dataZ, 'sng')

In [215]:
if __name__ == "__main__":
    main()
    print("Done XD")

Processing... 0.0 %
Processing... 6.25 %
Processing... 12.5 %
Processing... 18.75 %
Processing... 25.0 %
Processing... 31.25 %
Processing... 37.5 %
Processing... 43.75 %
Processing... 50.0 %
Processing... 56.25 %
Processing... 62.5 %
Processing... 68.75 %
Processing... 75.0 %
Processing... 81.25 %
Processing... 87.5 %
Processing... 93.75 %
Schemeing...
Scheme complete >:)
Bad Frame Ratio:  0.0
Done XD


In [131]:
def dataPatch(data):
    for ia in range(1,len(data)):
        if (np.sum(data[ia]) == 0) and (np.sum(data[ia - 1]) != 0) and (np.sum(data[ia + 1]) != 0):
            print('patched hole')
            data[ia] = np.mean(data[ia-1] + data[ia+1])
    return data